In [31]:
# 比较信噪比未知
import cv2
import numpy as np

image_noisy = cv2.imread('noisy_image.png', cv2.IMREAD_GRAYSCALE)

image_autocorr = np.fft.fftshift(np.fft.fft2(image_noisy * image_noisy.conj()))
noise_autocorr = np.abs(np.fft.fftshift(np.fft.fft2(image_noisy - np.mean(image_noisy)) * np.fft.fft2(image_noisy - np.mean(image_noisy)).conj()))
snr_estimated = np.max(image_autocorr) / np.max(noise_autocorr)

def wiener_filter(image, image_autocorr, noise_autocorr, snr_estimated):
    image_freq = np.fft.fftshift(np.fft.fft2(image))
    wiener_filter = np.conj(image_autocorr) / (image_autocorr + snr_estimated * noise_autocorr)
    restored_freq = wiener_filter * image_freq
    restored_image = np.real(np.fft.ifft2(np.fft.ifftshift(restored_freq)))
    return restored_image

restored_image = wiener_filter(image_noisy, image_autocorr, noise_autocorr, snr_estimated)
restored_image = np.clip(restored_image, 0, 255).astype(np.uint8)

cv2.imshow('Restored Image (Wiener Filter)', restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
# 信噪比已知
import cv2
import numpy as np

image_noisy = cv2.imread('noisy_image.png', cv2.IMREAD_GRAYSCALE)

snr_known = 3.0

noise_var = np.var(image_noisy) / (10 ** (snr_known / 10))

height, width = image_noisy.shape[:2]

cutoff_freq = 70
inverse_filter = np.ones((height, width), dtype=np.complex128)
inverse_filter[np.abs(np.fft.fftshift(np.fft.fftfreq(height, d=1.0/height))) > cutoff_freq] = 1e-6
inverse_filter[np.abs(np.fft.fftshift(np.fft.fftfreq(width, d=1.0/width))) > cutoff_freq] = 1e-6

def inverse_filtering(image, inverse_filter):
    image_freq = np.fft.fftshift(np.fft.fft2(image))
    restored_freq = image_freq * inverse_filter
    restored_image = np.real(np.fft.ifft2(np.fft.ifftshift(restored_freq)))
    return restored_image

restored_image = inverse_filtering(image_noisy, inverse_filter)
restored_image = np.clip(restored_image, 0, 255).astype(np.uint8)

cv2.imshow('Restored Image (Inverse Filtering)', restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


IndexError: boolean index did not match indexed array along dimension 0; dimension is 440 but corresponding boolean dimension is 466

In [33]:
#图像和噪声自相关函数已知
import cv2
import numpy as np
from scipy.signal import fftconvolve

image_noisy = cv2.imread('noisy_image.png', cv2.IMREAD_GRAYSCALE)

noise_autocorr = cv2.imread('noise_autocorr.png', cv2.IMREAD_GRAYSCALE)

snr_known = 3.0

noise_var = np.var(image_noisy) / (3 ** (snr_known / 3))

image_psd = np.abs(np.fft.fftshift(np.fft.fft2(image_noisy)))**2
noise_psd = np.abs(np.fft.fftshift(np.fft.fft2(noise_autocorr)))**2

wiener_filter = np.conj(image_psd) / (image_psd + noise_psd / noise_var)

def wiener_filtering(image, wiener_filter):
    image_freq = np.fft.fftshift(np.fft.fft2(image))
    restored_freq = wiener_filter * image_freq
    restored_image = np.real(np.fft.ifft2(np.fft.ifftshift(restored_freq)))
    return restored_image

restored_image = wiener_filtering(image_noisy, wiener_filter)
restored_image = np.clip(restored_image, 0, 255).astype(np.uint8)

cv2.imshow('Restored Image (Wiener Filter)', restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

IndexError: cannot do a non-empty take from an empty axes.